In [3]:
#FRONTEND CODE

import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder
import joblib
from scipy.stats import kurtosis, skew
import traceback

# Load the trained models and preprocessing tools
try:
    hash_model = load_model('/content/drive/My Drive/hash_model/hashing_algorithm_cnn_model.h5')
    hash_scaler = joblib.load('/content/drive/My Drive/hash_model/scaler.pkl')
    hash_label_encoder = joblib.load('/content/drive/My Drive/hash_model/label_encoder.pkl')

    stream_model = load_model('/content/drive/My Drive/stream_model/stream_algorithm_cnn_model.h5')
    stream_scaler = joblib.load('/content/drive/My Drive/stream_model/scaler.pkl')
    stream_label_encoder = joblib.load('/content/drive/My Drive/stream_model/label_encoder.pkl')

    block_model = load_model('/content/drive/My Drive/block_model/block_algorithm_cnn_model.h5')
    block_scaler = joblib.load('/content/drive/My Drive/block_model/scaler.pkl')
    block_label_encoder = joblib.load('/content/drive/My Drive/block_model/label_encoder.pkl')
except Exception as e:
    print("Error loading models or preprocessing tools.")
    print(e)
    traceback.print_exc()

# Function to extract features from the input text
def extract_features(cipher_text):
    try:
        cipher_bytes = bytes.fromhex(cipher_text)
        cipher_array = np.frombuffer(cipher_bytes, dtype=np.uint8)

        mean = np.mean(cipher_array)
        std_dev = np.std(cipher_array)

        # Histogram for entropy calculation
        histogram, _ = np.histogram(cipher_array, bins=256, range=(0, 256))
        histogram = histogram / np.sum(histogram)  # Normalize histogram
        entropy = -np.sum(histogram * np.log2(histogram + 1e-10))

        kurt = kurtosis(cipher_array)
        skewness = skew(cipher_array)

        min_value = np.min(cipher_array)
        max_value = np.max(cipher_array)
        range_value = max_value - min_value
        unique_byte_count = len(np.unique(cipher_array))

        # Combine features into a single list
        features = [
            mean, std_dev, entropy, kurt, skewness, min_value, max_value, range_value, unique_byte_count
        ]

        return features

    except Exception as e:
        print("Error extracting features from input.")
        print(e)
        traceback.print_exc()
        return None

# Function to preprocess and predict using a specified model
def predict(model, scaler, label_encoder, cipher_text):
    features = extract_features(cipher_text)
    if features is None:
        return None, "Error extracting features from input text."

    try:
        features = np.array(features).reshape(1, -1)  # Reshape for scaling
        features = scaler.transform(features)  # Scale features
        features = features.reshape(1, 9, 1, 1)  # Reshape for CNN (assuming single-channel images)

        # Predict using the model
        prediction = model.predict(features)
        confidence = prediction.max()
        predicted_label_index = np.argmax(prediction, axis=1)
        predicted_label = label_encoder.inverse_transform(predicted_label_index)[0]

        return confidence, predicted_label
    except Exception as e:
        print("Error in prediction process.")
        print(e)
        traceback.print_exc()
        return None, "Prediction failed."

# Define the function that integrates with Gradio
def predict_algorithm(cipher_text):
    hash_confidence, hash_prediction = predict(hash_model, hash_scaler, hash_label_encoder, cipher_text)
    stream_confidence, stream_prediction = predict(stream_model, stream_scaler, stream_label_encoder, cipher_text)
    block_confidence, block_prediction = predict(block_model, block_scaler, block_label_encoder, cipher_text)

    # Find the prediction with the highest confidence score
    confidences = [hash_confidence, stream_confidence, block_confidence]
    predictions = [hash_prediction, stream_prediction, block_prediction]

    if None in confidences:
        return "An error occurred during prediction."

    # Identify the prediction with the highest confidence score
    max_index = np.argmax(confidences)
    prediction = predictions[max_index]

    # Create an informative output string, listing all confidence values
    result = (
        f"Predicted Algorithm: {prediction}\n\n"
        f"Confidence Scores:\n"
        f" - Hashing Model: {hash_confidence:.4f} ({hash_prediction})\n"
        f" - Stream Cipher Model: {stream_confidence:.4f} ({stream_prediction})\n"
        f" - Block Cipher Model: {block_confidence:.4f} ({block_prediction})\n"
    )

    return result

# Update the Gradio interface
iface = gr.Interface(
    fn=predict_algorithm,
    inputs=gr.Textbox(label="Enter Cipher Text (Hexadecimal)"),
    outputs=gr.Textbox(label="Predicted Algorithm and Confidence Scores"),
    title="Cryptographic Algorithm Predictor",
    description="This tool predicts which cryptographic algorithm is most likely used for the provided cipher text. It uses separate models for Hashing, Stream Cipher, and Block Cipher algorithms."
)

iface.launch()


Error loading models or preprocessing tools.
[Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/My Drive/hash_model/hashing_algorithm_cnn_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


Traceback (most recent call last):
  File "<ipython-input-3-3c9b8db34f86>", line 13, in <cell line: 12>
    hash_model = load_model('/content/drive/My Drive/hash_model/hashing_algorithm_cnn_model.h5')
  File "/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_api.py", line 194, in load_model
    return legacy_h5_format.load_model_from_hdf5(
  File "/usr/local/lib/python3.10/dist-packages/keras/src/legacy/saving/legacy_h5_format.py", line 116, in load_model_from_hdf5
    f = h5py.File(filepath, mode="r")
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 561, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "/usr/local/lib/python3.10/dist-packages/h5py/_hl/files.py", line 235, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py/_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py/_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py/h5f.pyx", line 102, in h5py.

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://40216b0b2147c67dd9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
